We'll begin by using the keywords SELECT, FROM, and WHERE to get data from specific columns based on conditions you specify.

For clarity, we'll work with a small imaginary dataset pet_records which contains just one table, called pets.


## SELECT ... FROM¶
The most basic SQL query selects a single column from a single table. To do this,

    specify the column you want after the word SELECT, and then
    specify the table after the word FROM.

For instance, to select the Name column (from the pets table in the pet_records database in the bigquery-public-data project), our query would appear as follows:
```
SELECT Name
    FROM `bigquery-public-data.pet_records_pets`
```

Note that when writing an SQL query, the argument we pass to FROM is not in single or double quotation marks (' or "). It is in backticks (`).

## WHERE ...

BigQuery datasets are large, so you'll usually want to return only the rows meeting specific conditions. You can do this using the WHERE clause.

The query below returns the entries from the Name column that are in rows where the Animal column has the text 'Cat'.

In [1]:
from google.cloud import bigquery
import os

In [2]:
# Set this to the full absolute path of your downloaded key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/mwaki/Documents/Documents/Credentials/egerdrive-7433adb919ad.json"

In [3]:
# create a Client objec
client = bigquery.Client()
# construct a reference to the dataset
dataset_ref = client.dataset("openaq",project="bigquery-public-data")

In [4]:
# fetch the dataset
dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

global_air_quality


The dataset contains only one table, called global_air_quality. We'll fetch the table and take a peek at the first few rows to see what sort of data it contains

In [5]:
table_ref = dataset_ref.table('global_air_quality')
table = client.get_table(table_ref)
client.list_rows(table,max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


Everything looks good! So, let's put together a query. Say we want to select all the values from the city column that are in rows where the country column is 'US' (for "United States").

In [6]:
query = """select city from `bigquery-public-data.openaq.global_air_quality`
where country='US'
"""

## Submitting the query to the dataset

We're ready to use this query to get information from the OpenAQ dataset. As in the previous tutorial, the first step is to create a Client object.

In [8]:
query_job = client.query(query)

In [9]:
us_cities = query_job.to_dataframe()

/home/mwaki/miniconda3/envs/hood/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [10]:
# What five cities have the most measurements?
us_cities.city.value_counts().head()

city
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: count, dtype: int64

In [13]:
us_cities.value_counts()

city                                    
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
                                            ...  
Fort Walton Beach-Crestview-Destin              1
WILL                                            1
College Station-Bryan                           1
077                                             1
019                                             1
Name: count, Length: 826, dtype: int64

## Q&A: Notes on formatting

The formatting of the SQL query might feel unfamiliar. If you have any questions, you can ask in the comments section at the bottom of this page. Here are answers to two common questions:
Question: What's up with the triple quotation marks (""")?

**Answer:** These tell Python that everything inside them is a single string, even though we have line breaks in it. The line breaks aren't necessary, but they make it easier to read your query.
## Question: Do you need to capitalize SELECT and FROM?

**Answer:** No, SQL doesn't care about capitalization. However, it's customary to capitalize your SQL commands, and it makes your queries a bit easier to read.
